In [1]:
from geo_model2 import GeoModel
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Neighbourhood Influence only

In [ ]:
# simulate
seed = 187758
parameters = {"cost_clean": .4, "cost_dirty":.2, "base_output_dirty":0.4, "base_output_clean":0.2,
                 "metabolism_scalar_energy":1, "metabolism_scalar_money":1, "eta_global_trade":0.01,
                 "predisposition_decrease":0.000_05, "pareto_optimal":False, "seed":seed, "prob_neigh_influence" : 0}
x_probs = np.linspace(0,1,num=20)

welfares_test = np.empty(len(x_probs))
gini_vals = np.empty(len(x_probs))
modularity_vals = np.empty((len(x_probs)))
for idx, x in enumerate(x_probs):
    args = parameters.copy()
    args["prob_neigh_influence"] = x

    cur_model = GeoModel(**args) # 1000 runs
    cur_model.run_model(1024)
    data = cur_model.datacollector.get_model_vars_dataframe().iloc[-1]
    welfares_test[idx] = data.Welfare
    modularity_vals[idx] = data.modularity_ga
    gini_vals[idx] = data.Gini_welfare

/Users/paulhosek/PycharmProjects/Agent_based_modeling/country2.py:212: RuntimeWarning: divide by zero encountered in divide
  self.mrs = np.divide(np.multiply(np.add(self.w_energy, self.produced_energy), self.m_money),
/Users/paulhosek/PycharmProjects/Agent_based_modeling/country2.py:175: RuntimeWarning: invalid value encountered in power
  return np.power(self.w_energy + self.produced_energy + add_energy, self.m_energy / mt) \
/Users/paulhosek/PycharmProjects/Agent_based_modeling/country2.py:212: RuntimeWarning: divide by zero encountered in divide
  self.mrs = np.divide(np.multiply(np.add(self.w_energy, self.produced_energy), self.m_money),
/Users/paulhosek/opt/anaconda3/envs/Agent_based_modeling/lib/python3.10/site-packages/libpysal/weights/weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 5 disconnected components.
 There are 3 islands with ids: 1, 3, 17.
  warnings.warn(message)
/Users/paulhosek/opt/anaconda3/envs/Agent_based_modeling/lib/python3.

In [ ]:
# plot
plt.figure()
plt.plot(welfares_test)
plt.plot(modularity_vals)
plt.plot(gini_vals)